In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')
from sublimpy import tidy, utils
import pytz
import datetime as dt

seconds_in_timestep = 60*30
from metpy.constants import density_water

from metpy.units import units
import pint_pandas
from metpy import constants

# Open SOS Measurement Dataset, extract measurements

In [40]:
start_date = '20221130'
end_date = '20230509'
# open files
# tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
# tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_planar_fit.parquet')
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

## Add combined blowing snow flux variable
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue',
    'snow flux',
    1,
    'ue',
)

In [41]:
vars = [
    'dir_3m_ue', 'u_h2o__3m_ue', 'v_h2o__3m_ue', 'w_h2o__3m_ue',     'u_3m_ue',  'v_3m_ue',     'w_3m_ue',  'h2o_3m_ue', 
    'dir_10m_ue', 'u_h2o__10m_ue','v_h2o__10m_ue', 'w_h2o__10m_ue',   'u_10m_ue',  'v_10m_ue',  'w_10m_ue', 'h2o_10m_ue',
    'dir_3m_uw', 'u_h2o__3m_uw', 'v_h2o__3m_uw', 'w_h2o__3m_uw',     'u_3m_uw',  'v_3m_uw',     'w_3m_uw',  'h2o_3m_uw', 
    'dir_10m_uw', 'u_h2o__10m_uw','v_h2o__10m_uw', 'w_h2o__10m_uw',   'u_10m_uw',  'v_10m_uw',  'w_10m_uw', 'h2o_10m_uw',
    'dir_3m_d', 'u_h2o__3m_d', 'v_h2o__3m_d', 'w_h2o__3m_d',        'u_3m_d',   'v_3m_d',       'w_3m_d',   'h2o_3m_d',  
    'dir_10m_d', 'u_h2o__10m_d','v_h2o__10m_d', 'w_h2o__10m_d',      'u_10m_d',  'v_10m_d',     'w_10m_d',  'h2o_10m_d', 

    'dir_2m_c', 'u_h2o__2m_c', 'v_h2o__2m_c', 'w_h2o__2m_c',        'u_2m_c',   'v_2m_c',   'w_2m_c',   'h2o_2m_c',   'specifichumidity_2m_c',    'airdensity_2m_c',
    'dir_3m_c', 'u_h2o__3m_c', 'v_h2o__3m_c', 'w_h2o__3m_c',        'u_3m_c',   'v_3m_c',   'w_3m_c',   'h2o_3m_c',   'specifichumidity_3m_c',    'airdensity_3m_c',
    'dir_5m_c', 'u_h2o__5m_c', 'v_h2o__5m_c', 'w_h2o__5m_c',        'u_5m_c',   'v_5m_c',   'w_5m_c',   'h2o_5m_c',   'specifichumidity_5m_c',    'airdensity_5m_c',
    'dir_10m_c', 'u_h2o__10m_c','v_h2o__10m_c', 'w_h2o__10m_c',      'u_10m_c',  'v_10m_c', 'w_10m_c',  'h2o_10m_c',  'specifichumidity_10m_c',   'airdensity_10m_c',
    'dir_15m_c', 'u_h2o__15m_c', 'v_h2o__15m_c', 'w_h2o__15m_c',     'u_15m_c',  'v_15m_c', 'w_15m_c',  'h2o_15m_c',  'specifichumidity_15m_c',   'airdensity_15m_c',
    'dir_20m_c', 'u_h2o__20m_c', 'v_h2o__20m_c', 'w_h2o__20m_c',     'u_20m_c',  'v_20m_c', 'w_20m_c',  'h2o_20m_c',  'specifichumidity_20m_c',   'airdensity_20m_c',

    'T_1m_c', 'T_2m_c', 'T_3m_c', 'T_4m_c', 'T_5m_c', 'T_6m_c', 'T_7m_c', 'T_8m_c', 'T_9m_c', 'T_10m_c', 
    'T_11m_c', 'T_12m_c', 'T_13m_c', 'T_14m_c', 'T_15m_c', 'T_16m_c', 'T_17m_c', 'T_18m_c', 'T_19m_c', 'T_20m_c',

    'Rsw_in_9m_d', 'Rnet_9m_d',

    'SF_avg_ue',

    'Ri_3m_c',

    'L_3m_c',
]

pivot_df = tidy_df[tidy_df.variable.isin(vars)][
    ['time', 'variable', 'value']
].pivot_table(index='time', columns='variable', values='value')

assert len(vars) == len(pivot_df.columns.unique())

In [42]:
HEIGHT = 3

w_tavg_savg = 0.25*(pivot_df[f'w_{HEIGHT}m_ue'] + pivot_df[f'w_{HEIGHT}m_uw'] + pivot_df[f'w_{HEIGHT}m_c'] + pivot_df[f'w_{HEIGHT}m_d'])
q_tavg_savg = 0.25*(pivot_df[f'h2o_{HEIGHT}m_ue'] + pivot_df[f'h2o_{HEIGHT}m_uw'] + pivot_df[f'h2o_{HEIGHT}m_c'] + pivot_df[f'h2o_{HEIGHT}m_d'])

w_star_ue = pivot_df[f'w_{HEIGHT}m_ue'] - w_tavg_savg
w_star_uw = pivot_df[f'w_{HEIGHT}m_uw'] - w_tavg_savg
w_star_c = pivot_df[f'w_{HEIGHT}m_c'] - w_tavg_savg
w_star_d = pivot_df[f'w_{HEIGHT}m_d'] - w_tavg_savg

q_star_ue = pivot_df[f'h2o_{HEIGHT}m_ue'] - q_tavg_savg
q_star_uw = pivot_df[f'h2o_{HEIGHT}m_uw'] - q_tavg_savg
q_star_c = pivot_df[f'h2o_{HEIGHT}m_c'] - q_tavg_savg
q_star_d = pivot_df[f'h2o_{HEIGHT}m_d'] - q_tavg_savg

term1 = 0.25*(pivot_df[f'w_h2o__{HEIGHT}m_ue'] + pivot_df[f'w_h2o__{HEIGHT}m_uw'] + pivot_df[f'w_h2o__{HEIGHT}m_c'] + pivot_df[f'w_h2o__{HEIGHT}m_d'])
term2 = 0.25*(
    w_star_ue*q_star_ue + w_star_uw*q_star_uw + w_star_c*q_star_c + w_star_d*q_star_d
)
term3 = 0.25*(
    pivot_df[f'w_{HEIGHT}m_ue'] + pivot_df[f'w_{HEIGHT}m_uw'] + pivot_df[f'w_{HEIGHT}m_c'] + pivot_df[f'w_{HEIGHT}m_d']
)*0.25*(
    pivot_df[f'h2o_{HEIGHT}m_ue'] + pivot_df[f'h2o_{HEIGHT}m_uw'] + pivot_df[f'h2o_{HEIGHT}m_c'] + pivot_df[f'h2o_{HEIGHT}m_d']
)

In [43]:
pivot_df['term1 (covariance flux)'] = term1
pivot_df['term2 (stationary eddy flux)'] = term2
pivot_df['term3 (advective flux)'] = term3

In [57]:

alt.Chart(
    pivot_df.loc['20230201': '20230203'].reset_index()
).transform_fold([
    'term1 (covariance flux)',
    'term2 (stationary eddy flux)',
    'term3 (advective flux)'
]).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q"),
    alt.StrokeDash("key:N"),
    alt.Color('month:N').sort([12,1,2,3,4])
).properties(height = 300, width=425, title=[f'{HEIGHT}-meter spatially averaged vertical moisture transport', 'by different mechanisms, monthly composites'])

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [51]:
src = pivot_df.groupby([pivot_df.index.month, pivot_df.index.floor('30T').time]).mean().reset_index()
src['month'] = src['time']
src['time_of_day'] = src['level_1'].apply(lambda time: dt.datetime.combine(dt.date(2024,1,1), time))

alt.Chart(src[src.month.isin([12,1,2,3,4])]).transform_fold([
    'term1 (covariance flux)',
    'term2 (stationary eddy flux)',
    'term3 (advective flux)'
]).mark_line().encode(
    alt.X('time_of_day:T'),
    alt.Y("value:Q"),
    alt.StrokeDash("key:N"),
    alt.Color('month:N').sort([12,1,2,3,4])
).properties(height = 300, width=425, title=[f'{HEIGHT}-meter spatially averaged vertical moisture transport', 'by different mechanisms, monthly composites'])

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_94673/3091067853.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src = pivot_df.groupby([pivot_df.index.month, pivot_df.index.floor('30T').time]).mean().reset_index()


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [44]:
src = pivot_df.groupby([pivot_df.index.month, pivot_df.index.floor('30T').time]).mean().reset_index()
src['month'] = src['time']
src['time_of_day'] = src['level_1'].apply(lambda time: dt.datetime.combine(dt.date(2024,1,1), time))

alt.Chart(src[src.month.isin([12,1,2,3,4])]).transform_fold([
    'term1 (covariance flux)',
    'term2 (stationary eddy flux)',
    'term3 (advective flux)'
]).mark_line().encode(
    alt.X('time_of_day:T'),
    alt.Y("value:Q"),
    alt.Color("key:N"),
    alt.Facet('month:O', columns=3).sort([12,1,2,3,4])
).properties(height = 150, width=225, title='Spatially averaged vertical moisture transport by different mechanisms, monthly composites ')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_94673/3752400284.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src = pivot_df.groupby([pivot_df.index.month, pivot_df.index.floor('30T').time]).mean().reset_index()


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
